## はじめに
+ 「ランタイム」＞「すべてのセルを実行」により、ダミーデータで表示されるようになっています。
  + <a href="https://colab.research.google.com/drive/1JL_tI76mkgF4mloVYSYxDwzQRtAPs2IN">こちらからGoogleColab上で試すことが可能</a>です（実行にはGoogleアカウントが必要）。
+ このファイルは、GoogleColabでの動作を想定し、インデント=半角空白2文字にしてます。
+ 配布前に、このMarkdownセルは削除してください。

# このファイルを配布前にすべきこと
+ 参加者がGoogleアカウントを使えるようにしておく
+ 全国の人口データ、自分の街の人口データを、ダミーデータ1や2の形式で作成し、URLを取得。アクセス権限を参加者のGoogleドライブから開けるように変更。
  + データの形式はダミーデータ<a href="https://docs.google.com/spreadsheets/d/1j5qLDVR_FhHyT7mouvxngsVQbMjVtl6LGcwoj44futI">1</a>や<a href="https://docs.google.com/spreadsheets/d/10raUmLtk93kUGyD9fUPJpZVi9va-13jtr2Y5u7XP4ZI">2</a>をご覧ください。
  + 0.get_japan_from_estat.ipynbによって出力されるcsvにより、全国のデータがcsvで出力されます。Googleスプレッドシート形式に変換
    + 0.get_japan_from_estat.ipynb自体も教材にしてもいいですが、estatのidが必要です（ユーザー登録など）。
  + csvのまま使う、Excelにしてから使う、などをしたい場合は、このプログラムの読み込み部分を変えてください。
+ 以下をカスタマイズ
  + このファイルの「ある街」を、使用する街の名前に変更する
  + 人口データが対応する年に合わせて、「2012-2022」を変更（2箇所）
  + 「そのまま」指定時の出力単位やデータを変更（現状は、「ある街」の総人口が数万-十数万人として「全国：千人、ある街：人」に設定）
  + 「全国人口データのGoogleスプレッドシートのURL」「ある街の人口データのGoogleスプレッドシートのURL」の双方に、上で取得したURLを貼る。
+ 「編集」＞「出力をすべて消去」によって出力を消し、アクセス権限を参加者のGoogleドライブから開けるようにする（閲覧のみにする）

# 最初に「ファイル」＞「コピーをドライブに保存」を実行しましょう（そうしないと、今日の作業内容が保存されません）

In [ ]:
pip install japanize_matplotlib

In [ ]:
# ここを実行すると、Googleドライブにアクセスしていいかの許可を求められます。
# 許可して先へ進めてください。
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# 人口データ
e-Statから読み込んだデータ（全国）と、ある街のデータを、Googleドライブから読み込む

In [ ]:
pop={}
##全国のデータに変更（現在はダミー1）
url="https://docs.google.com/spreadsheets/d/1j5qLDVR_FhHyT7mouvxngsVQbMjVtl6LGcwoj44futI"
xlsx = gc.open_by_url(url)
for f in xlsx.worksheets(): pop["全国"] = pd.DataFrame(xlsx.worksheet(f.title).get_all_records())

##ある街のデータに変更（現在はダミー2）
url="https://docs.google.com/spreadsheets/d/10raUmLtk93kUGyD9fUPJpZVi9va-13jtr2Y5u7XP4ZI"
xlsx = gc.open_by_url(url)
for f in xlsx.worksheets(): pop["ある街"] = pd.DataFrame(xlsx.worksheet(f.title).get_all_records())

In [ ]:
pop["全国"]

In [ ]:
pop["ある街"]

In [ ]:
# データの一部を取り出す
years = [2019,2020,2022]#2012-2022まで指定可能
ages = [5,12]#0-99まで指定可能
danjo = ["男"]##男，女，男女計，が指定可能

print("全国（単位：千人）")
area_="全国"
display(pop[area_][pop[area_].year.isin(years) &
                   pop[area_].age.isin(ages) &
                   pop[area_].sex.isin(danjo)])
print("ある街（単位：人）")
area_="ある街"
display(pop[area_][pop[area_].year.isin(years) &
                   pop[area_].age.isin(ages) &
                   pop[area_].sex.isin(danjo)])

# グラフ表示

In [ ]:
# グラフを表示する関数
def view_graph(danjo, area, base, years):
  plt.figure(figsize=(10,5))###画像の大きさ（縦横）を決める
  lw=2##線の太さ

#縮尺を，今回表示する最初の年の値から，決定する
  scale0 = {}
  captions = []
  for danjo_ in danjo:
    for area_ in area:
      if base=="そのまま":
        scale0[area_+danjo_], captions = 100, ["全国：千人、ある街：人"]####人口規模によって調整
      else:
        if area_ in pop:
          p = pop[area_][(pop[area_].year==min(years)) & (pop[area_].sex==danjo_)]
        else:
          print('「全国」か「ある街」だけが「area」に指定できます！打ち間違いをチェックしてみてください！')
          return
        scale0[area_+danjo_], caption0 = return_scale(p, base, area_, min(years))
        captions.append(area_+danjo_+"："+caption0)

# グラフの出力部分
  for danjo_ in danjo:
    for area_ in area:
      for year in years:
        ## データの選択
        p = pop[area_][(pop[area_].year==year) & (pop[area_].sex==danjo_)]
        ## グラフを表示
        plt.plot(p["age"], 100* p["value"]/scale0[area_+danjo_],
                 linewidth=lw, label="'"+str(year)[2:]+area_[0]+danjo_)

## グラフ全体の見栄え
  plt.title("\n".join(captions), fontsize=23)
  plt.xlim([-19,103])##敢えて，x軸方向に表示を広げ，legendがグラフの邪魔をしないように
  plt.xticks(fontsize=17)#x軸メモリ
  plt.xlabel("年齢", fontsize=17)
  plt.yticks(fontsize=17)#y軸メモリ
  plt.ylabel("人数", fontsize=17)
  plt.legend(fontsize=17, handlelength=1, framealpha=0.3)#後2つは，線の長さと，透明度
  plt.show()

In [ ]:
def return_scale(data0, base0, area0, year0):
  if area0=="全国": tani0="千人"
  else: tani0="人"
  if base0=="平均値":#縮尺を「平均値」にしたとき
    value0 = data0["value"].mean(axis=0)
    return value0, "'"+str(year0)[2:]+base0+str(value0)+tani0+"=100"
  elif base0=="中央値":#縮尺を「中央値」にしたとき
    value0 = data0["value"].median(axis=0)
    ##中央値の計算がどうやっているかをprint出力するためのプログラム
    value49 = sorted(data0.value)[49]
    value50 = sorted(data0.value)[50]
    age49 = data0.loc[data0.value==value49, "age"]
    age50 = data0.loc[data0.value==value50, "age"]
    if list(age49)==list(age50):##49,50番目が同じだった場合（0,1,...,99番目のうち中央値は49,50番目の平均）
      ages = ",".join([str(k_) for k_ in age49])+"歳の" + str(value0)+tani0
    else:
      ages = "("+",".join([str(k_) for k_ in age49])+"歳の" + str(value49)+tani0+"+"
      ages += ",".join([str(k_) for k_ in age50])+"歳の" + str(value50) +tani0+")/2="+str(value0)+tani0
    print(ages)
    ##中央値の計算出力終わり
    return value0, "'"+str(year0)[2:]+base0+str(value0)+tani0+"=100"
  elif base0=="最大値":#縮尺を「最大値」にしたとき
    value0 = data0["value"].max(axis=0)
    ###最大値がどこかを取得
    agemax = data0.loc[data0.value==value0, "age"]
    ages = ",".join([str(k_) for k_ in agemax])+"歳"
    return value0, "'"+str(year0)[2:]+base0+ages+str(value0)+tani0+"=100"
  elif base0=="合計値":#縮尺を「合計値」にしたとき
    value0 = data0["value"].sum(axis=0)
    return value0, "'"+str(year0)[2:]+base0+str(value0)+tani0+"=100"
  else:
    print("グラフの縮尺の指示がおかしいよ！とりあえず「そのまま」で出します")
    return 100, "（縮尺の指示ミス！→→「"+base0+"」を変えてください）"

In [ ]:
danjo = ["女"]##男・女・男女計，が使える．例：["男","女"]
area = ["全国","ある街"]##全国・ある街，が使える．
base = "そのまま"##縮尺を，そのまま・平均値・中央値・最大値・合計値，から1つだけ選ぶ．
years = [2021,2022]##出力年（2012-2022）をここで選ぶ
view_graph(danjo, area, base, years)

In [ ]:
##複数のグラフを出す＆一度表示したグラフの設定を残す，がしたい場合は，セルを自由に増やしてください．


